In [ ]:
import pandas as pd
import nltk

In [ ]:
dataset1 = pd.read_csv('/content/drive/MyDrive/project/Sentence_Score.csv')
dataset1 = dataset1.iloc[:, 1:]
dataset1.head(1)

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   

                                      Clean_sentence  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   

   font_style_score  sentence_position  thematic_score  sentence_similarity  
0          0.032258                1.0        0.121429             0.351307

In [ ]:
dataset2 = pd.read_csv("/content/drive/MyDrive/project/Summary.csv")
dataset2 = dataset2.iloc[:, 1:]
dataset2.head(1)

Docno   Dir                                       tf-idf_score  \
0  AP880926-0203  d30e  Officials of the seven key industrialized nati...   

                                        adverb_score  \
0  ``It is ... sad to note that we, American, Asi...   

                                          verb_score  \
0  ``In a market place where one group of operato...   

                                     adjective_score  \
0  Officials of the seven key industrialized nati...   

                                     text_rank_score  \
0  Brazil, with $121 billion in foreign loans, ha...   

                                    font_style_score  \
0     Third World countries led by Brazil, thewor...   

                                   sentence_position  \
0     Third World countries led by Brazil, thewor...   

                                      thematic_score  \
0     Third World countries led by Brazil, thewor...   

                                 sentence_similarity  
0     Third World countries led by Brazil, thewor...

In [ ]:
w1 = 0.33
w2 = 0.33
w3 = 0.33
def make_combination(lst):
  m1 = lst[0] 
  m2 = lst[1] 
  m3 = lst[2] 
  final_score = m1*w1 + m2*w2 + m3*w3
  return final_score
dataset1['combined_score'] = dataset1[['adjective_score', 'sentence_position', 'thematic_score']].apply(make_combination, axis = 1)

In [ ]:
def generate_combine_summary(lst):
  dir = lst[0]
  doc = lst[1]
  temp_df = dataset1[(dataset1['Dir'] == dir) & (dataset1['Docno'] == doc)].sort_values(by = 'combined_score', ascending = False)
  rs = ''
  cnt = 0
  for i in temp_df['sentence']:
    if cnt + len(i.split()) <= 100:
      cnt += len(i.split())
      rs += (i+' ')
  return rs
dataset2['combined_score'] = dataset2[['Dir', 'Docno']].apply(generate_combine_summary, axis = 1)

In [ ]:
dataset2.head(2)

Docno   Dir                                       tf-idf_score  \
0   AP880926-0203  d30e  Officials of the seven key industrialized nati...   
1  WSJ910607-0063  d30e  The Christian Children's Fund ofRichmond, Va.,...   

                                        adverb_score  \
0  ``It is ... sad to note that we, American, Asi...   
1  Trickle Up, which makes thousands of $100 loan...   

                                          verb_score  \
0  ``In a market place where one group of operato...   
1  David Munro, an officialwith TechnoServe of No...   

                                     adjective_score  \
0  Officials of the seven key industrialized nati...   
1     Tiny Third World businesses commonly repay ...   

                                     text_rank_score  \
0  Brazil, with $121 billion in foreign loans, ha...   
1     Tiny Third World businesses commonly repay ...   

                                    font_style_score  \
0     Third World countries led by Brazil, thewor...   
1  The Christian Children's Fund ofRichmond, Va.,...   

                                   sentence_position  \
0     Third World countries led by Brazil, thewor...   
1     Tiny Third World businesses commonly repay ...   

                                      thematic_score  \
0     Third World countries led by Brazil, thewor...   
1  Trickle Up, which makes thousands of $100 loan...   

                                 sentence_similarity  \
0     Third World countries led by Brazil, thewor...   
1  Undaunted, the women switched to chickens and ...   

                                      combined_score  
0     Third World countries led by Brazil, thewor...  
1     Tiny Third World businesses commonly repay ...

In [ ]:
dfr1 = pd.read_csv('/content/drive/MyDrive/project/rouge-1_score.csv')
dfr1 = dfr1.iloc[:, 1:]
dfr2 = pd.read_csv('/content/drive/MyDrive/project/rouge-2_score.csv')
dfr2 = dfr2.iloc[:, 1:]
dfrl = pd.read_csv('/content/drive/MyDrive/project/rouge-l_score.csv')
dfrl = dfrl.iloc[:, 1:]
reference_dataset=pd.read_csv("/content/drive/MyDrive/project/reference_summary_2001.csv")
reference_dataset=reference_dataset.iloc[:,[3, 4]]

In [ ]:
dataset3 = dataset2[['Dir', 'Docno', 'combined_score']]
dataset3 = pd.merge(dataset3, reference_dataset, on='Docno')

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_rouge_score(lst):
  dir = lst[0]
  doc = lst[1]
  ref_text = lst[-1]
  rg = (rouge.get_scores(lst[2], ref_text))[0]
  return pd.Series([rg['rouge-1']['r'], rg['rouge-1']['p'], rg['rouge-1']['f'], 
                    rg['rouge-2']['r'], rg['rouge-2']['p'], rg['rouge-2']['f'],
                    rg['rouge-l']['r'], rg['rouge-l']['p'], rg['rouge-l']['f']])

In [ ]:
dataset3[['combine-rouge-1-r', 'combine-rouge-1-p', 'combine-rouge-1-f',
          'combine-rouge-2-r', 'combine-rouge-2-p', 'combine-rouge-2-f',
          'combine-rouge-l-r', 'combine-rouge-l-p', 'combine-rouge-l-f']] = dataset3.apply(get_rouge_score, axis = 1)

In [ ]:
dataset4 = pd.DataFrame()

In [ ]:
dic= {}
for c in dataset3.iloc[:,4:].columns:
  dic[c] = dataset3[c].mean()

In [ ]:
dic

{'combine-rouge-1-r': 0.29834157814198947,
 'combine-rouge-1-p': 0.2849595637496149,
 'combine-rouge-1-f': 0.28871225758477703,
 'combine-rouge-2-r': 0.11148661086989312,
 'combine-rouge-2-p': 0.10752193373780813,
 'combine-rouge-2-f': 0.10844989590058568,
 'combine-rouge-l-r': 0.27829759710159846,
 'combine-rouge-l-p': 0.2658330293768884,
 'combine-rouge-l-f': 0.26932661098236804}

In [ ]:
def set_combine_score(method):
  lst =[dic[k] for k in list(dic.keys()) if method in k]
  return pd.Series(lst)

In [ ]:
final_dataset = pd.read_csv('/content/drive/MyDrive/project/final_score.csv')
final_dataset[['combine_r', 'combine_p', 'combine_f']] = final_dataset['method'].apply(set_combine_score)
final_dataset.to_csv('/content/drive/MyDrive/project/final_score_comb.csv')

In [ ]:
final_dataset.head(1)

Unnamed: 0   method  tf-idf_score_r  tf-idf_score_p  tf-idf_score_f  \
0           0  rouge-1        0.271961        0.253818         0.26031   

   adverb_score_r  adverb_score_p  adverb_score_f  verb_score_r  verb_score_p  \
0        0.255005        0.243161        0.246325      0.263208      0.250262   

   ...  sentence_position_f  thematic_score_r  thematic_score_p  \
0  ...             0.288581          0.298636          0.291116   

   thematic_score_f  sentence_similarity_r  sentence_similarity_p  \
0           0.29266               0.265736               0.255271   

   sentence_similarity_f  combine_r  combine_p  combine_f  
0               0.257484   0.298342    0.28496   0.288712  

[1 rows x 32 columns]